In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data for price prediction model.csv')

In [3]:
df

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,0.0,10.0,1.20,3,3,3.0,3.0,1145.000000,0,0,0,0.0,0.0
1,0.0,114.0,1.25,4,3,3.0,2.0,1994.777045,1,0,0,1.0,1.0
2,0.0,91.0,0.86,2,2,2.0,1.0,1186.324045,0,0,1,2.0,0.0
3,0.0,42.0,1.35,3,2,4.0,1.0,1378.931760,0,0,0,1.0,0.0
4,0.0,96.0,2.21,3,3,4.0,3.0,1866.283075,1,0,2,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,0.0,11.0,1.15,2,2,3.0,3.0,1300.935993,0,0,0,1.0,1.0
3581,0.0,11.0,0.36,2,2,2.0,3.0,733.708470,0,0,0,1.0,2.0
3582,0.0,101.0,1.49,4,5,4.0,3.0,2017.000000,1,0,1,2.0,2.0
3583,0.0,103.0,1.31,2,2,4.0,3.0,1432.682751,0,0,0,2.0,2.0


In [81]:
# one hot encoding on categorical columns
x = df.drop(columns = ['price'])
y = df['price']

In [82]:
x

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,0.0,10.0,3,3,3.0,3.0,1145.000000,0,0,0,0.0,0.0
1,0.0,114.0,4,3,3.0,2.0,1994.777045,1,0,0,1.0,1.0
2,0.0,91.0,2,2,2.0,1.0,1186.324045,0,0,1,2.0,0.0
3,0.0,42.0,3,2,4.0,1.0,1378.931760,0,0,0,1.0,0.0
4,0.0,96.0,3,3,4.0,3.0,1866.283075,1,0,2,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3580,0.0,11.0,2,2,3.0,3.0,1300.935993,0,0,0,1.0,1.0
3581,0.0,11.0,2,2,2.0,3.0,733.708470,0,0,0,1.0,2.0
3582,0.0,101.0,4,5,4.0,3.0,2017.000000,1,0,1,2.0,2.0
3583,0.0,103.0,2,2,4.0,3.0,1432.682751,0,0,0,2.0,2.0


In [83]:
y

0       1.20
1       1.25
2       0.86
3       1.35
4       2.21
        ... 
3580    1.15
3581    0.36
3582    1.49
3583    1.31
3584    1.35
Name: price, Length: 3585, dtype: float64

In [4]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [85]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [86]:
# Applying the log1p transformation to the target variable 
# Bcz price was right skewed
y_transformed = np.log1p(y)

In [105]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom','bathroom','built_up_area','servant room','store room']),
        ('cat', OneHotEncoder(drop = 'first', handle_unknown = 'ignore'), columns_to_encode)
    ],
    remainder = 'passthrough'
)    

In [107]:
# Creating a pipeline

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel = 'rbf'))
])    

In [109]:
# Kfold cross validation
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')

/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [110]:
scores.mean()

0.8841832027934154

In [113]:
scores.std()

0.018981951235336303

In [115]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y_transformed, test_size = 0.2, random_state = 42)

In [117]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [119]:
y_pred = pipeline.predict(x_test)

In [121]:
y_pred = np.expm1(y_pred)

In [125]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test), y_pred)

0.5668923983795152